In [1]:
import pandas as pd
import json

# Load Excel sheets
dashboard = pd.ExcelFile("data.xlsx")
data = pd.read_excel(dashboard, "Statistics")
aes = pd.read_excel(dashboard, 'AEs')
referrals = pd.read_excel(dashboard, 'Referrals')

partnerDistricts =pd.read_excel("partnerDistrict.xlsx")

# Fill missing values and convert types
data.fillna(0, inplace=True)
aes.fillna(0, inplace=True)
referrals.fillna(0, inplace=True)
try:
    if type(referrals['ReferrelRecordingYear'])!= int:
        referrals['ReferrelRecordingYear'] = 2025
except:
    pass
try:
    data['year'] = data['year'].astype(int)
    data['recordingMonth'] = data['recordingMonth'].astype(str)
    aes['AERecordingYear'] = aes['AERecordingYear'].astype(int)
    aes['AERecordingMonth'] = aes['AERecordingMonth'].astype(str)
    referrals['ReferrelRecordingYear'] = referrals['ReferrelRecordingYear'].astype(int)
    referrals['ReferrelRecordingMonth'] = referrals['ReferrelRecordingMonth'].astype(str)
except:
    pass



# Process rows and build JSON
def process_row(row):
    try:
        row_data = row.to_dict()
        if int(pd.to_numeric(row.get('totalAES', 0), errors='coerce')) > 0:
            recordedAEs = aes[
                (aes['AERecordingSite'] == row['facilityName']) &
                (aes['AERecordingMonth'] == row['recordingMonth']) &
                (aes['AERecordingYear'] == row['year'])
            ]
    
            recordedAEs = recordedAEs[recordedAEs['ae_classification']!=0]
    
            row_data['matchingAES'] = recordedAEs.to_dict(orient='records')
        else:
            row_data['matchingAES'] = []
                
        if int(pd.to_numeric(row.get('referralToOtherServices', 0), errors='coerce')) > 0:
            otherReferrals = referrals[
                (referrals['ReferrelRecordingSite'] == row['facilityName']) &
                (referrals['ReferrelRecordingMonth'] == row['recordingMonth']) &
                (referrals['ReferrelRecordingYear'] == row['year'])
            ]
    
            otherReferrals = otherReferrals[otherReferrals['services_referred']!=0]
            # print(otherReferrals)
            row_data['otherReferrals'] = otherReferrals.to_dict(orient='records')
        else:
            print("no data")
            row_data['otherReferrals'] = []
    
        return row_data
    except:
        pass

partner_lookup = dict(zip(partnerDistricts['DISTRICT'], partnerDistricts['PARTNER']))

data['Partner'] = data['District'].map(partner_lookup)


json_data = data.apply(process_row, axis=1).tolist()

# Save to JSON
output_file = "data.json"
with open(output_file, "w") as file:
    json.dump(json_data, file, indent=4)

# Print the JSON
print(json.dumps(json_data, indent=4))



no data
no data
[
    {
        "AERecordingMonth": "April",
        "AERecordingSite": "Beula",
        "AERecordingYear": 2025,
        "District": "MATOBO",
        "ReferrelRecordingMonth": 0,
        "ReferrelRecordingSite": 0,
        "ReferrelRecordingYear": 0,
        "Site_Name": 0,
        "TotalMCsBYMethod": 11,
        "aeComments": "antibiotic and health education",
        "ae_classification": "Moderate",
        "ae_type_code": "C IN 2",
        "ageGroupsChange": 0,
        "checkMonthValidation": 1,
        "checkYearValidation": 1,
        "checkifFacilityHasBeenSaved": 0,
        "circumcising_cadre": "Nurse",
        "client_age": 22,
        "date_ae_identified": "2025-04-04",
        "facilityName": "Beula",
        "followUpTotal": 11,
        "formType": "submission",
        "fu15-19": 0,
        "fu20-24": 6,
        "fu25-29": 5,
        "fu30-34": 0,
        "fu35-39": 0,
        "fu40-44": 0,
        "fu45-49": 0,
        "fu50": 0,
        "fuAgeUpdate": 0

In [2]:
data

,AERecordingMonth,AERecordingSite,AERecordingYear,District,ReferrelRecordingMonth,ReferrelRecordingSite,ReferrelRecordingYear,Site_Name,TotalMCsBYMethod,aeComments,...,total_mcs_referred_to_other_services,total_surgicalDisposable,total_surgicalReusable,totalhivNegative,totalhivNegativeNC,uncircumcisedClientsForHTS,vmmc_number,year,facilityType,Partner
0,April,Beula,2025,MATOBO,0,0,0,0,11,antibiotic and health education,...,0,0,0,11,0,0,2025M00009,2025,Outreach,ZICHIRE
1,0,0,0,MATOBO,0,0,0,0,7,0,...,0,OK,OK,7,1,1,0,2025,Outreach,ZICHIRE
2,0,0,0,CHITUNGWIZA,0,0,0,0,239,0,...,0,OK,OK,194,0,1,0,2025,0,ZICHIRE
